### Load Pickle file

In [ ]:
import pickle
from pathlib import Path

root_module = Path.cwd()
ext = ".pkl"
name = "deep_sea_ensemble"
load_dir = root_module.parent.joinpath("data/")
file_dir = load_dir.joinpath(name + ext)
data = pickle.load(open(file_dir, "rb"))

### Get metrics from raw data

In [ ]:
import numpy as np
from scipy.stats import sem

data["mean_regret"] = {}
data["ci_regret"] = {}
data["mean_solve_time"] = {}
for key in data["total_regret"].keys():
    # Get mean and standard error of total regret
    data["mean_regret"][key] = np.mean(data["total_regret"][key], axis=-1)
    data["ci_regret"][key] = sem(data["total_regret"][key], axis=-1)

### Plots

In [ ]:
%matplotlib widget
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
from ube_mbrl.utils.plot import PARAMS

plt.rcParams.update(PARAMS)

# Get unique agent types and deep sea sizes in the data
keys_arr = np.array(list((data["total_regret"].keys())), dtype=object)

# convert agent types to array so we can sort it and order it how we like
agent_types = np.array(list(set(keys_arr[:, 0])), dtype=tuple)
agent_types = agent_types[agent_types[:,1].argsort()]
# rearrange to desire order 
order = [1, 2, 0]
agent_types = agent_types[order].tolist()
# convert back to tuple so we use it as index
agent_types = [tuple(agent_type) for agent_type in agent_types]

ensemble_sizes = np.sort(list(set(keys_arr[:, 1].astype(int))))
deep_sea_sizes = np.sort(list(set(keys_arr[:, -1])))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(7.5, 2.0),gridspec_kw={'wspace':0.05,'hspace':0.5})

ax_dict = {}
for size, ax in zip(deep_sea_sizes, axes.flatten()):
    ax_dict[size] = ax

colors = {}
cmap = plt.get_cmap("tab10")
for i, agent_type in enumerate(agent_types):
    colors[agent_type] = cmap(i)

wd = 0.2
pos = [-wd, 0, wd]
patches = []
xticks = list(range(1, len(ensemble_sizes) + 1))
xticks_labels = set()
for size in deep_sea_sizes:
    ax_dict[size].set_title(fr"$L = {{{size}}}$")
    for i, agent in enumerate(agent_types):
        for ensemble_size, xtick in zip(ensemble_sizes, xticks):
            xticks_labels.add(str(ensemble_size))
            idx = (agent, str(ensemble_size), size)
            ax_dict[size].bar(
                xtick + pos[i],
                data["mean_regret"][idx][-1], 
                yerr=data["ci_regret"][idx][-1], 
                width=wd, 
                color=colors[agent]
            )

patches = []
for agent in agent_types:
    agent_name, uq_method = agent
    if uq_method == "exact_ube_3":
        label = r"\texttt{ofu-exact-ube}" + " (ours)"
    elif uq_method == "ensemble":
        label = r"\texttt{ofu-ensemble-var}"
    else:
        label = fr"\texttt{{{agent_name}-}}\texttt{{{uq_method}}}"
    patches.append(mpatches.Patch(color=colors[agent], label=label))


axes[0].legend(handles=patches, loc = 'lower center', ncol=4, bbox_to_anchor=(1.0, -0.5), frameon=False)

axes[1].yaxis.set_ticklabels([])
axes[0].set_ylabel("Total regret")


# Manual legend needed for bar plot with multiple bars per tick
labels = np.sort(np.array(list(xticks_labels), dtype=int))
for _, ax in ax_dict.items():
    ax.set_xticks(ticks=xticks, labels=labels)
    ax.set_xlabel("Ensemble size")
    ax.set_ylim(bottom=0, top=800)

### Save figures

In [ ]:
# Save figures
import os
import pickle
from pathlib import Path
root_module = Path.cwd()
fig_dir = root_module.parent.joinpath(f"figures/deep_sea_ensemble_size_ablation.pdf")
fig.savefig(fig_dir, bbox_inches="tight", transparent=False)

# License

>Copyright (c) 2023 Robert Bosch GmbH
>
>This program is free software: you can redistribute it and/or modify <br>
>it under the terms of the GNU Affero General Public License as published<br>
>by the Free Software Foundation, either version 3 of the License, or<br>
>(at your option) any later version.<br>
>
>This program is distributed in the hope that it will be useful,<br>
>but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
>GNU Affero General Public License for more details.<br>
>
>You should have received a copy of the GNU Affero General Public License<br>
>along with this program.  If not, see <https://www.gnu.org/licenses/>.